In [2]:

import random
import numpy as np

B = 3                    #tipos de veiculos b
R = 10                     #conjunto de rotas r
FT = [141, 531, 35]            #frota disponível por tipo de veiculo b
PD = [2928, 2885, 2968, 2972, 2965, 2879, 2884, 2845, 2953, 2888]           #demanda de passageiros por hora na rota r
CAP = [21, 37, 60]               #capacidade de passageiros para cada tipo de veiculo b
frequencia = np.zeros((B,R))    #frequência de serviço do tipo de veículo b na rota r 
Q = np.zeros((B,R))              #número de passageiros alocados em cada rota r pelos diferentes tipos de veículos b

CO = [2.8, 3.75, 3.57]         #custo operacional (motorista, gasolina e manutenção) para cada veículo do tipo b
CE = 4.05                     #custo de espera na rota r
CD = 0.18                       #custo de desconforto para passageiros     
T = 60                    #período de planejamento (60 min)
H = np.zeros((B,R))    #intervalo de tempo entre as partidas do tipo de veículo b na rota r 

rotas = []        #lista de rotas restantes
S = 10           #número de soluções geradas
solucoes = []   #Lista para armazenar informações das soluções geradas

#imprimindo os parâmetros
print("\n Rotas = ", R, "\n Tipo de veiculo = ", B)
print(" Frota = ", FT, "\n Demanda de passageiros = ", PD, "\n Capacidade do veiculo = ", CAP, "\n")
print(" Custo Operacional = ", CO, "\n Custo de Espera = ", CE, "\n Custo de Desconforto = ", CD, "\n Periodo = ", T, "\n")

def calculaFO(CustoOperacional, CustoEspera, CustoDesconforto):
    return (CustoOperacional + CustoEspera + CustoDesconforto)

#imprimir as soluções
def imprimirSolucao(s, B, R, frequencia, Q, FO):
    print("_"*48)
    print("Solucao ", s+1, "\n")
    #para cada tipo de veículo b
    for b in range(B):
        print("_Tipo ", b)
        #para cada rota r
        for r in range(R):
            print("rota ", r+1, ": ", "frequencia: ", frequencia[b][r],"  -  qntd_passageiros: ", Q[b][r] )
        print()
    print("Funcao Objetivo: ", FO)

def distribuir_passageiros(frequencia, Q, PD, CAP):
    B, R = frequencia.shape
    for r in range(R):
        passageiros_restantes = PD[r]
        for b in range(B):
            if frequencia[b][r] > 0:
                capacidade_disponivel = CAP[b] * frequencia[b][r]
                passageiros_atendidos = min(passageiros_restantes, capacidade_disponivel)
                Q[b][r] = passageiros_atendidos
                passageiros_restantes -= passageiros_atendidos

    
#gerando as soluções
def gerarSolucoes(B, R, FT, PD, CAP, CO, CE, CD, T, H):
   
    #variável auxiliar pd(demanda de passageiros) recebe a demanda de passageiros por hora na rota r
    pd = PD.copy()
    #variável auxiliar cap(capacidade) recebe a capacidade de passageiros para cada tipo de veiculo b
    c = CAP.copy()
    
    FO = 0                  #função objetivo
    somaCustoOperacional=0  #soma do custo operacional para cada tipo b
    somaCustoEspera=0       #soma do custo de espera na rota r
    somaCustoDesconforto=0     #soma do custo do desconforto
    
    #para cada tipo de veículo b
    for b in range(B):
        #número máximo de veículos disponíveis para cada tipo de veículo b 
        max_v = FT[b]
        
        custoOperacional = 0 #custo operacional para cada tipo b
        custoEspera = 0      #custo de espera na rota r
        custoDesconforto = 0    #custo do desconforto
        
        #para cada rota r
        for r in range (R):
            #adiciona o conjunto de rotas na lista de rotas restantes
            rotas.append(r)
            
        #enquanto houver rotas restantes
        while(rotas):
            #sorteia uma rota r
            r = random.choice(rotas)  
            #pega um valor que esteja entre a capacidade de passageiros e a demanda de passageiros
            #min_value = min(c[b],pd[r])
            #sorteia um número de veículo enviado para a rota r 
            frequencia[b][r] = random.randint(0,max_v)
            
            #passageiros_restantes = p[r]
            
            #calcula o custo operacional
            custoOperacional = custoOperacional  + (CO[b] * frequencia[b][r])
            
            #se nenhum veículo foi enviado para rota r, 
            if frequencia[b][r] == 0:
                #a demanda de passageiros será igual a zero
                Q[b][r] = 0
                #e o intervalo de tempo entre as partidas do tipo de veículo b na rota r, também será igual a zero
                H[b][r]=0
                
                #custo de espera também será igual a zero
                custoEspera = 0
                
                #remove a rota r da lista de rotas restantes
                rotas.remove(r)
                continue
                
            #se o número de veículo enviado para rota r for diferente de zero,
            #então calcula-se o H_br (intervalo de tempo entre as partidas do tipo de veículo b na rota r)
            H[b][r] = T/frequencia[b][r]
            
            #calcula o custo de espera
            custoEspera = custoEspera + (CE * H[b][r] + CD)
            
            c[b] = CAP[b] * frequencia[b][r]
            
            #pega o menor valor entre a demanda de passageiros e a capacidade de passageiros
            passageiros_atendidos = min(pd[r], c[b])
            #sorteia um número de passageiros para cada tipo de veículo b na rota r
            Q[b][r] = random.randint(0,passageiros_atendidos)
            #passageiros_restantes -= passageiros_atendidos
            
            #calculo do custo do desconforto
            custoDesconforto = custoDesconforto + ((frequencia[b][r] * CAP[b]) - Q[b][r] )
            
            #atualiza a capacidade de passageiros b
            c[b] -= Q[b][r]
            #atualiza a demanda de passageiros da rota r
            pd[r] -= Q[b][r]
            #atualiza o número maximo de veículos disponíveis para cada tipo b 
            max_v -= frequencia[b][r]
            #remove a rota r da lista de rotas restantes
            rotas.remove(r)
            
        somaCustoOperacional = somaCustoOperacional + custoOperacional #soma-se o custo operacional
        somaCustoEspera = somaCustoEspera + custoEspera             #soma-se o custo do custo de espera
        somaCustoDesconforto = somaCustoDesconforto + custoDesconforto   #soma-se o custo do desconforto 
        
    
    #calcula a função objetivo
    FO = calculaFO(somaCustoOperacional, somaCustoEspera, somaCustoDesconforto)
    
    #imprimindo as soluções 
    imprimirSolucao(s, B, R, frequencia, Q, FO)
   
    return FO, frequencia


#para cada solução s
for s in range(S): 
    FO, frequencia = gerarSolucoes(B, R, FT, PD, CAP, CO, CE, CD, T, H)




 Rotas =  10 
 Tipo de veiculo =  3
 Frota =  [141, 531, 35] 
 Demanda de passageiros =  [2928, 2885, 2968, 2972, 2965, 2879, 2884, 2845, 2953, 2888] 
 Capacidade do veiculo =  [21, 37, 60] 

 Custo Operacional =  [2.8, 3.75, 3.57] 
 Custo de Espera =  4.05 
 Custo de Desconforto =  0.18 
 Periodo =  60 

________________________________________________
Solucao  1 

_Tipo  0
rota  1 :  frequencia:  0.0   -  qntd_passageiros:  0.0
rota  2 :  frequencia:  2.0   -  qntd_passageiros:  21.0
rota  3 :  frequencia:  0.0   -  qntd_passageiros:  0.0
rota  4 :  frequencia:  1.0   -  qntd_passageiros:  8.0
rota  5 :  frequencia:  1.0   -  qntd_passageiros:  13.0
rota  6 :  frequencia:  0.0   -  qntd_passageiros:  0.0
rota  7 :  frequencia:  0.0   -  qntd_passageiros:  0.0
rota  8 :  frequencia:  0.0   -  qntd_passageiros:  0.0
rota  9 :  frequencia:  137.0   -  qntd_passageiros:  303.0
rota  10 :  frequencia:  0.0   -  qntd_passageiros:  0.0

_Tipo  1
rota  1 :  frequencia:  221.0   -  qntd_pass